# Radioactive Decay Interactives

This notebook has embedded in it code for interactive investigation of radioative decay.  

Specifically, there are two interactives:
- A Radioactive Decay model showing decay of a population of atoms over time.
- A geochron dating model

In [1]:
from IPython.display import display
import numpy as np
import bqplot as bq
import ipywidgets as widgets
import random as random
import pandas as pd
import number_formatting as nf
from math import ceil, floor, log10

In [8]:
## Originally developed June 2018 by Samuel Holen

## Pre-construct model of radioactive decay of a population
## of parent and daughter atoms.
## 

# Constants Related to decay of the parent species to the daughter species
N_parent = 900          # initial number of parent atoms (should be a perfect square)
N_daughter = 0          # initial number of daughter atoms
tau = 1                 # placeholder for the half-life of the parent species 
h = 1.0                 # time step  
mu = np.log(2.) / tau   # constant for decay time distribution 

# Initialize tracking of number of atoms
Parent_counts = []          # list of number of parent atoms 
Dauther_counts = []          # list of number of daughter atoms

# Generate a uniform random distribution of N_parent numbers from 0 to 1
z = np.random.rand(N_parent)

# Function to convert uniform distribution of random numbers to
# a distribution weighted to model radiactive decay. Unsorted representing the
# decay of each object.
decay_times = -np.log(1 - z) / mu
decay_times_sorted = np.sort( decay_times )

# Genereate array of numbers of atoms left
# Adjusted so that each count contains 0 and N_parent
Parent_counts = np.arange(N_parent,-1, -1, dtype='int')      # Number of parent atoms
Daughter_counts = np.ones_like(Parent_counts)
Daughter_counts = N_parent - Parent_counts   # Number of daughter atoms

#
# Construct Pandas data frames
#

# Time column adjusted to include t=0
decay_data = pd.DataFrame()
decay_data['time'] = np.concatenate((np.zeros(1),decay_times_sorted))
decay_data['Parent'] = Parent_counts
decay_data['Daughter'] = Daughter_counts

# Data array for species
species = pd.DataFrame()
species['parent_long'] = ['Thallium','Uranium','Rubidium','Carbon','Generic']
species['daughter_long'] = ['Lead','Thorium','Strontium','Nitrogen','Generic']
species['parent_short'] = ['Tl-208','U-235','Rb-87','C-14','Parent']
species['daughter_short'] = ['Pb-208','Th-231','Sr-87','N-14','Daughter']
species['half-lives'] = [3.053 * 60, 703.8, 48.8, 5730, tau]  
species['timeunits'] = ['seconds', 'million years', 'billion years', 'years', 'half-lives']

In [9]:
##
## Set up counts versus time plot
## **Default set to Tl-208 for everthing**

# Set up axes
x_time = bq.LinearScale(min = 0, max=max(decay_times*species['half-lives'][0]))
y_number = bq.LinearScale(min = 0, max=N_parent)
y_fraction = bq.LinearScale(min = 0, max=1)

# Forces the number of ticks to be 6 and to start at 0
tick_vals = np.linspace(0, max(decay_times*species['half-lives'][0]),6)

# Labels and scales for Axes
ax_x_time = bq.Axis(label=species['timeunits'][0], scale=x_time,
                    num_ticks=6, tick_values=tick_vals)
ax_y_number = bq.Axis(label='Number of atoms', scale=y_number, orientation='vertical')
ax_y_fraction = bq.Axis(label='Fraction of atoms', scale=y_fraction, orientation='vertical')

# Define tooltip (not working)
def_tt_parent = bq.Tooltip(fields=['x', 'y'], formats=['.2f', '.2f'], labels=['time', species['parent_short'][0]])
def_tt_daughter = bq.Tooltip(fields=['x', 'y'], formats=['.2f', '.2f'], labels=['time', species['daughter_short'][0]])

# Define the lines
line_parent = bq.Lines(x=decay_data['time']/species['half-lives'][0], y=[decay_data['Parent'][0]], 
                       scales={'x': x_time, 'y': y_number}, display_legend=True, colors=['red'], 
                       labels=[species['parent_short'][0]], tooltip=def_tt_parent)
line_daughter = bq.Lines(x=decay_data['time']/species['half-lives'][0], y=[decay_data['Daughter'][0]], 
                         scales={'x': x_time, 'y': y_number}, display_legend=True, colors=['blue'], 
                         labels=[species['daughter_short'][0]] ,tooltip=def_tt_daughter)

# Creates figure for plot
fig_counts = bq.Figure(axes=[ax_x_time, ax_y_number], marks=[line_parent, line_daughter], 
                       legend_location='right', legend_style={'fill': 'white'}, 
                       title='Counts versus Time', background_style={'fill': 'black'}, 
                       layout={'width': '500px', 'min_height': '400px'},
                      animation=1000)


In [10]:
# Slider widget to control the figures, controls the amount of time that has passed
Time_slide = widgets.FloatSlider(
    value=0.,
    description='Time',
    min=0.,
    max=max(decay_times*species['half-lives'][0])+1,
    step=h,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.1f'
)

# Widget to display the number of parent atoms present
parent_present = widgets.Text(
    value = str(N_parent),
    style = {'description_width': 'initial'},
    #description = species['parent_short'][0]+' remaining',
    disabled = True   
)
# Widget to display the number of daughter atoms present
daughter_present = widgets.Text(
    value = str(0),
    style = {'description_width': 'initial'},
    #description = species['daughter_short'][0]+' produced',
    disabled = True   
)
# Widgets to label the time slider with units
Time_label = widgets.Label(value=str(Time_slide.value))
unit_label = widgets.Label(value=str(species['timeunits'][0]))

# Labels for the parent/daughter present displays
parent_label = widgets.Label(value=species['parent_short'][0]+' remaining')
daughter_label = widgets.Label(value=species['daughter_short'][0]+' produced')

# Checkbox to choose whether to display the number of each species
# or the fraction of each
frac_or_num = widgets.Checkbox(value=False, description='Display as fractions')

# Widget to allow one to choose which species to work with
pick_Species = widgets.RadioButtons(options=species['parent_long'][:], 
                                 value='Thallium', description='Species:', disabled=False,
                                 layout=widgets.Layout(align_content='center', align_items='center', 
                                          display='flex', 
                                          flex_flow='column', height='150px', max_height='200px', 
                                          max_width='300px', min_height='100px', min_width='125px', 
                                          overflow_x='hidden', overflow_y='hidden', width='175px'))

# Scale for population figure
x_sc = bq.LinearScale(min=1, max=np.sqrt(N_parent))
y_sc = bq.LinearScale(min=1,max=np.sqrt(N_parent))

# Axes for population figure
ax_x = bq.Axis(scale=x_sc, num_ticks=0)
ax_y = bq.Axis(scale=y_sc, orientation='vertical', num_ticks=0)

# Creates an array of x values: [1,2,...,30,1,2,...30,.....,1,2,...,30]
x_ls = []
for i in range(1,int(np.sqrt(N_parent))+1):
    x_ls.append(float(i))
x_ls = x_ls * int(np.sqrt(N_parent))
x_arr = np.array(x_ls)

# Creates an array of y values: [1,1,...,1,2,2,...2,......,30,30,...,30]
y_ls = []
for i in range(1,int(np.sqrt(N_parent))+1):
    y_ls += [float(i)] * int(np.sqrt(N_parent))
y_arr = np.array(y_ls)    

# Creates a color array with the same number of entries as the number of atoms in
# the sample
Colors = ['red'] * N_parent

In [11]:
# Function to update the plots in response to the controllable widgets
def Update(change=None):
    # Determine index of matching species
    new_index = species.loc[species.parent_long == pick_Species.value].index[0]
   
    # Sets the half-life of the selected species
    hf = species['half-lives'][new_index]
    
    # Sets the max value of the time slider to the final decay time of the selected species
    Time_slide.max = max(decay_times)*hf+1
    
    # Changes timestep based on whether or not the generic species is selected.
    if pick_Species.value == 'Generic':
        Time_slide.step = h/10
    else:
        Time_slide.step = h
    
    # Changes the color of the correct number of decayed species, randomly distributed
    for i in range(N_parent):
        if Time_slide.value >= decay_times[i]*hf:
            Colors[i] = 'blue'
        else:
            Colors[i] = 'red'
    
    time_arr = hf * decay_data['time']
    
    # Update the parent and daughter plots
    i = 0
    while i < N_parent + 1 and hf*decay_data['time'][i] < Time_slide.value:        
        i += 1
    if i > 0:
        i -= 1
        
    # Update the plotting arrays    
    daughter_decay = decay_data['Parent'][0:i+1]
    parent_decay = decay_data['Daughter'][0:i+1] 
    
    # Apply the color change
    population_scat.colors = Colors
    
    line_parent.x = time_arr
    line_daughter.x = time_arr

     
    # Updates the time slider label/units
    if Time_slide.value < 1e3 or pick_Species.value == 'Thallium':
        unit = species['timeunits'][new_index]
        unit_label.value = unit
        Time_label.value = str(Time_slide.value)  
    elif pick_Species.value == 'Uranium' and Time_slide.value >= 1e3:
        unit = str('billion years')
        unit_label.value = unit
        time = nf.exp2LaTeX(Time_slide.value/1000,3)[0]
        Time_label.value = time
    elif pick_Species.value == 'Carbon' and Time_slide.value >= 1e3:
        unit = str('thousand years')
        unit_label.value = unit
        time = nf.exp2LaTeX(Time_slide.value/1000,3)[0]
        Time_label.value = time
    else:
        unit = species['timeunits'][new_index]
        unit_label.value = unit   
        Time_label.value = str(round(Time_slide.value))

        
    # Updates the x-axis
    x_time.max = max(decay_times*hf)
    tick_vals = np.linspace(0, max(decay_times*hf),6)
    ax_x_time.tick_values = tick_vals
    ax_x_time.scale = x_time
    ax_x_time.label = unit
    fig_counts.axes = [ax_x_time,ax_y_number]     
    
    # Update the units and value displayed on the slider
    if frac_or_num.value == False:
        parent_present.value = str(decay_data['Parent'][i])
        daughter_present.value = str(decay_data['Daughter'][i])

        # Update the x and y arrays for the parent and daughter lines
        line_parent.y = parent_decay
        line_daughter.y = daughter_decay
        line_parent.scales={'x': x_time, 'y': y_number}
        line_daughter.scales={'x': x_time, 'y': y_number}
        fig_counts.marks = [line_parent,line_daughter]

    else:
        # Fraction mode enabled
        fig_counts.axes = [ax_x_time,ax_y_fraction]
        # Update the x and y arrays for the parent and daughter lines
        line_parent.y = (1/N_parent)*parent_decay
        line_daughter.y = (1/N_parent)*daughter_decay
        line_parent.scales={'x': x_time, 'y': y_fraction}
        line_daughter.scales={'x': x_time, 'y': y_fraction}
        fig_counts.marks = [line_parent,line_daughter]
        parent_present.value = '{:.3f}'.format((1/N_parent)*decay_data['Parent'][i])
        daughter_present.value = '{:.3f}'.format((1/N_parent)*decay_data['Daughter'][i])
    
    # Update the legend
    parent_label_new = species['parent_short'][new_index]
    daughter_label_new = species['daughter_short'][new_index]
    line_parent.labels = [parent_label_new]
    line_daughter.labels = [daughter_label_new]
    
    # Update the species in the box that shows how many are present    
    parent_label.value = parent_label_new + ' produced'
    daughter_label.value = daughter_label_new + ' remaining'
    
    # Update the tooltip labels
    line_parent.tooltip.labels[1] = parent_label_new
    line_daughter.tooltip.labels[1] = daughter_label_new


# Function and variable to reset the time slider to 0 whenever a new species is selected.
x = pick_Species.index    
def Reset(change=None):
    global x
    if pick_Species.index != x:
        Time_slide.value = 0
        x = pick_Species.index
    




In [12]:
# Plot the population model
population_scat = bq.Scatter(x=x_arr, y=y_arr, scales={'x': x_sc, 'y': y_sc}, colors =['red'])

# Update the values/colors
pick_Species.observe(Reset, names=['value'])
Time_slide.observe(Reset, names=['value'])
Time_slide.observe(Update, names=['value'])
parent_present.observe(Update, names=['value'])
daughter_present.observe(Update, names=['value'])
pick_Species.observe(Update, names=['value'])
Time_label.observe(Update, names=['value'])
parent_label.observe(Update, names=['value'])
daughter_label.observe(Update, names=['value'])
frac_or_num.observe(Update, names=['value'])


# Figure for the population
fig = bq.Figure(title='Population', marks=[population_scat], axes=[ax_x, ax_y], 
                background_style={'fill' : 'black'},padding_x = 0.025,
                min_aspect_ratio=1, max_aspect_ratio=1)

# Boxes to organize display
slide_box = widgets.HBox([Time_slide, Time_label,unit_label])
parent_box = widgets.HBox([parent_label,parent_present])
daughter_box = widgets.HBox([daughter_label,daughter_present])
parent_box.children[0].layout.width = '50%'
daughter_box.children[0].layout.width = '50%'
slide_check_box = widgets.VBox([slide_box, frac_or_num])

value_box = widgets.VBox([parent_box,daughter_box])
value_box.layout.width = '80%'
species_box = widgets.HBox([value_box, pick_Species])

top_box = widgets.HBox([fig_counts, fig])
top_box.children[0].layout.width = '50%'
top_box.children[1].layout.width = '50%'

bottom_box = widgets.HBox([species_box, slide_check_box])
bottom_box.children[0].layout.width = '50%'
bottom_box.children[1].layout.width = '50%'

# Final display
Final = widgets.VBox([top_box,bottom_box])
Final.layout.overflow = 'hidden'
display(Final)

In [14]:
line_parent.tooltip.labels

['time', 'Rb-87']

In [21]:
# Define tooltip (not working)
def_tt_parent = bq.Tooltip(fields=['x', 'y'], formats=['.2f', '.2f'], labels=line_parent.tooltip.labels)
def_tt_daughter = bq.Tooltip(fields=['x', 'y'], formats=['.2f', '.2f'], labels=line_daughter.tooltip.labels)

In [22]:
line_parent.tooltip = def_tt_parent
line_daughter.tooltip = def_tt_daughter

In [20]:
line_parent.x

array([  0.00000000e+00,   5.39448089e-02,   1.04562899e-01,
         1.98509479e-01,   2.14267912e-01,   4.75912446e-01,
         4.75943185e-01,   5.73619530e-01,   6.24361837e-01,
         6.77099632e-01,   7.11837644e-01,   8.09488877e-01,
         8.43527513e-01,   8.44254109e-01,   9.38407474e-01,
         9.56552129e-01,   1.07187810e+00,   1.10507303e+00,
         1.23953982e+00,   1.24294636e+00,   1.34679277e+00,
         1.35503463e+00,   1.42857960e+00,   1.48733327e+00,
         1.64708517e+00,   1.73126418e+00,   1.78881498e+00,
         1.78933570e+00,   1.83722043e+00,   1.84765236e+00,
         1.91536073e+00,   1.97037516e+00,   2.15630843e+00,
         2.18696636e+00,   2.23501541e+00,   2.26479033e+00,
         2.28254573e+00,   2.32271800e+00,   2.32909889e+00,
         2.45019800e+00,   2.47960776e+00,   2.59163939e+00,
         2.66924379e+00,   2.69044483e+00,   2.74231515e+00,
         2.86051513e+00,   2.88901241e+00,   3.13489471e+00,
         3.35276420e+00,

# Geochrons

Assuming a non-radiogenic isotope (that is, an isotope that is not the result of radioactive decay) that also will not decay, its amount should be constant.  This means that for different mineral samples we can measure the ratio of parent isotope versus the non-radiogenic isotope ($P/D_i$) and daughter isotope ($D$) versus the non-radiogenic isotope ($D/D_i$) to build an isochron plot.  For example, using the following isotopes

- $D_i$ (non-radiogenic isotope of daughter element)
- $D$ (Daughter Isoptope)
- $P$ (Parent isotope)

an isochron plot could plot $D/D_i$ versus $P/D_i$.  

What sets the *geochron method* (also known as the *isochron method*) apart from the just measuring parent and daughter abundances is the use of the non-radiogenic isotope of the daughter element.  This avoids the assumption of no initial daughter isotope before the rock solidified (radioactive decay can occur while rock is molten).

Some minerals in the rock incoprorate parent better than daughter which is why the initial amount of parent 
isotope versus daughter isotope can vary.  We expect daughter versus non-radiogenic isotope ratio to be constant
if we pick the non-radiogenic isotope to be the same element as the daughter isotope.

With all this said, it is actually often not this simple as many daughter isotopes are themselves radioactive and decay, leading to a chain of reactions, so comparing abundances of parent to daughter isotopes is not simple.

*Note:* The idea for the geochron dating interactive came from a Isochron Diagram Java app at *ScienceCourseware.org*.  However that app had some issues in that it didn't divide by a non-radiogenic isotope (or at least didn't mention it).  In fact, they used $D_i$ for the initial amount of daughter isotope instead of the non-radiogenic isotope of the same element as the daughter isotope.

In [ ]:
##
## Define the various isotopes we want to consider
##

isotope_info = pd.DataFrame(columns=['Name', 'PName', 'PAbbrev', 'DName', 'DAbbrev', 'DiName', 'DiAbbrev', 'HalfLife', 'HLUnits'])
isotope_info['index'] = ['generic', 'Rb87']
isotope_info['Name'] = ['Generic', 'Rb-87->Sr-87']
isotope_info['PName'] = ['Parent', 'Rubidium-87']
isotope_info['PAbbrev'] = ['P', 'Rb-87']
isotope_info['DName'] = ['Daughter', 'Strontium-87']
isotope_info['DAbbrev'] = ['D', 'Sr-87']
isotope_info['DiName'] = ['Non-Radiogenic Isotope of Daughter Element', 'Strontium-86']
isotope_info['DiAbbrev'] = ['D_i', 'Sr-86']
isotope_info['HalfLife'] = [ 1, 48.8 ]
isotope_info['HLUnits'] = [ 'half-lives', 'Billion years']
isotope_info = isotope_info.set_index('index')

# Set initial isotope to plot
init_isotope = 'generic'

In [ ]:
##
## Define the initial amounts of parent and daughter in the sample.
##
## In principle, I would change this depending on the isotopes we plot.  But I am only plotting
## Rb87 --> Sr-87, since that is the most classical use of this Geochron approach.
##

# Range of P to D_i fractions and initial amounts of D to D_i to consider
P2Di_min = 0.05
P2Di_max = 0.40
D2Di0_min = 0.05
D2Di0_max = 0.75

# Generate three mineral samples in different thirds of the entire range
range_P2Di  = (P2Di_max-P2Di_min)

# Create sample amounts
n_samples = 4
nums = np.array(list(range(1, n_samples+1)))
initial_samples = pd.DataFrame(index=nums)
initial_D2Di0 = D2Di0_min + (D2Di0_max - D2Di0_min) * np.random.random()
initial_samples['P2Di'] = P2Di_min + (range_P2Di/n_samples) * (nums - np.random.random(n_samples))
initial_samples['D2Di'] = initial_D2Di0*np.ones_like(nums)


In [ ]:
##
## Define functions to call when building interactive plot
##

def amt_left(sample_in, taus):
    # Generate a sample DataFrame after tau half-lifes given an initial DataFrame
    sample = sample_in.copy(deep = True)
    sample['P2Di'] = sample_in['P2Di']*((1/2)**(taus))
    sample['D2Di'] = sample_in['D2Di'] + sample_in['P2Di']*(1 - (1/2)**(taus))
    return sample

def line_points(sample):
    global x_min, x_max, y_min, y_max, initial_D2Di0
    
    # Determine the end points of a line going through the sample points.
    x_range = x_max - x_min
    y_range = y_max - y_min
    
    # Slope (extrapolate from first two points - could be done by a fit to the points)
    slope = (sample['D2Di'][2]-sample['D2Di'][1])/(sample['P2Di'][2]-sample['P2Di'][1])
    y_final = initial_D2Di0 + slope*x_range
    x_points = (x_min, x_max)
    y_points = (initial_D2Di0, y_final)
    return x_points, y_points, slope

def init2current(samples0, samples):
    # Compute the lines connecting initital and final points for plotting
    n_pts = len(samples0)

    xlist = []
    ylist = []
    for pt in range(1, n_pts+1):
        x = np.array([ samples0['P2Di'][pt], samples['P2Di'][pt] ])
        y = np.array([ samples0['D2Di'][pt], samples['D2Di'][pt] ])
        xlist.append(x)
        ylist.append(y)
    
    return(xlist, ylist)
    
def HL_changed(change):
    global isotope, sample, initial_samples, dots_current, line_current, connectors, slope_label
    
    # Determine half-life of this isotope
    idx = (isotope_info.Name == isotope.value)
    HL = float(isotope_info[idx].HalfLife.tolist()[0])
    
    # How many half-lives have passed?  Use this to get new sample and line info
    this_tau = HL_slider.value / HL
    sample = amt_left(initial_samples, this_tau)
    x_sample, y_sample, slope =  line_points(sample)
    
    
    # Update plot
    dots_current.x = sample['P2Di']
    dots_current.y = sample['D2Di']
    line_current.x = x_sample
    line_current.y = y_sample
    slope_label.value = 'Slope: {0:.2f}'.format(slope)
    xlist, ylist = init2current(initial_samples, sample)
    connectors.x = xlist
    connectors.y = ylist
    
    
def isotope_changed(change):
    global ax_x_P2Di, ax_y_D2Di, HL_slider, HLlabel, UnitsText, Max_half_lives

    # Extract the necessary isotope descriptors from the Pandas DataFrame
    idx = (isotope_info.Name == change.new)
    HL = float(isotope_info[idx].HalfLife.tolist()[0])
    HLUnits = isotope_info[idx].HLUnits.tolist()[0]
    PAbbrev = isotope_info[idx].PAbbrev.tolist()[0]
    DAbbrev = isotope_info[idx].DAbbrev.tolist()[0]
    DiAbbrev = isotope_info[idx].DiAbbrev.tolist()[0]

    # Get old half-life
    idx_old = (isotope_info.Name == change.old)
    HL_old = float(isotope_info[idx_old].HalfLife.tolist()[0])

    # Determine current age reading from slider and adjust to new units
    init_age = HL_slider.value 
    
    # Hard code generic versus others
    if (change.new != isotope_info.loc['generic'].Name):
        HL_slider.description = "Time"
    else: 
        HL_slider.description = "Half-lives"    

    # Adjust time scales
    if (HL_old < HL):
        # Adjust maximum limits first before adjusting values (since new HL > old HL)
        HL_slider.max = Max_half_lives*HL
        HLlabel.max = HL_slider.max 
        HL_slider.value = HL*(init_age/HL_old)
        HLlabel.value = HL_slider.value       
    else:
        # Adjust maximum limits after adjusting values (since new HL < old HL)
        HL_slider.value = HL*(init_age/HL_old)
        HLlabel.value = HL_slider.value
        HL_slider.max = Max_half_lives*HL
        HLlabel.max = HL_slider.max 
                
    # Set the axes and other labels to display
    UnitsText.value = HLUnits
    ax_x_P2Di.label = '{0} / {1}'.format(PAbbrev, DiAbbrev)
    ax_y_D2Di.label = '{0} / {1}'.format(DAbbrev, DiAbbrev)



In [ ]:
##
## Set up isochron plot
##

# Largest possible fraction of decay (only go out to 5 half-lives)
Max_half_lives = 5
Max_decay_fraction = 1 - (1/2)**(Max_half_lives)

# detemine maximum and minimum values of X and Y axes
x_step = 0.05
x_min = 0
x_max = x_step * ceil(initial_samples['P2Di'][n_samples] / x_step)
y_step = 0.04
y_min = y_step * floor(initial_D2Di0 / y_step)
y_max = y_step * ceil((initial_D2Di0 + initial_samples['P2Di'][n_samples] * Max_decay_fraction) / y_step)

# Labels and scales for Axes
x_P2Di = bq.LinearScale(min = x_min, max = x_max)
y_D2Di = bq.LinearScale(min = y_min, max = y_max)
ax_x_P2Di = bq.Axis(label='P / D_i', scale=x_P2Di)
ax_y_D2Di = bq.Axis(label='D / D_i', scale=y_D2Di, orientation='vertical')

# Set up initial conditions
taus = 0    # zero half lives past
sample = amt_left(initial_samples, taus)

##
## Define the lines
##

# Initial amount of daughter line (with dots for initial amounts of parent)
x_init, y_init, slope_init =  line_points(initial_samples)
line_initial = bq.Lines(x=x_init, y=y_init, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='dashed', colors=['red'], labels=['Initial Sample'])
dots_initial = bq.Scatter(x=initial_samples['P2Di'], y=initial_samples['D2Di'], scales={'x': x_P2Di, 'y': y_D2Di}, 
                   colors=['white'], stroke='red', fill= True, labels=['Initial Isochron'])

# Current quantities on isochron line
x_sample, y_sample, slope =  line_points(sample)
line_current = bq.Lines(x=x_sample, y=y_sample, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='solid', colors=['red'], labels=['Current Isochron'])
dots_current = bq.Scatter(x=sample['P2Di'], y=sample['D2Di'], scales={'x': x_P2Di, 'y': y_D2Di}, 
                   colors=['red'], stroke='red', fill= True, labels=['Current Isochron'])

# Connect Initial and Current quantities on isochron line
xlist, ylist = init2current(initial_samples, sample)
connectors = bq.Lines(x=xlist, y=ylist, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='dotted', colors=['black'])

##
## Construct plot
##
isochron = bq.Figure(axes=[ax_x_P2Di, ax_y_D2Di], 
                     marks=[connectors, line_initial, dots_initial, line_current, dots_current],
                     title='Geochron Diagram', 
                     layout={'width': '700px', 'height': '500px', 
                             'max_width': '700px', 'max_height': '500px',
                             'min_width': '600px', 'min_height': '400px'})

##
## Construct controls
##

# Select Generic or Specific Isotopes
isotope = widgets.RadioButtons(options=list(isotope_info.Name), 
                               value=isotope_info.loc[init_isotope].Name, description='Isotope:', 
                               disabled=False, 
                               layout=widgets.Layout(height='75px', max_height='100px', min_height='50px', 
                                                    width='200px', max_width='300px',  min_width='100px'))
isotope.observe(isotope_changed, 'value')

# Slider and text field controling age
idx = (isotope_info.Name == isotope.value)
HL = float(isotope_info[idx].HalfLife.tolist()[0])
HLUnits = isotope_info[idx].HLUnits.tolist()[0]

HL_slider = widgets.FloatSlider(value=0, min=0, max=Max_half_lives*HL, step=0.02,
                                description='Half-lives', disabled=False,
                                continuous_update=True, orientation='horizontal',
                                readout=False, readout_format='.2f',
                                layout=widgets.Layout(height='75px', max_height='100px', min_height='50px', 
                                                    width='200px', max_width='300px',  min_width='100px'))
HL_slider.observe(HL_changed, 'value')

# Get units value and units for age label, then apply them
HLlabel = widgets.BoundedFloatText(value = HL_slider.value, min = HL_slider.min, max = HL_slider.max, 
                                   step = HL_slider.step,
                                       layout={'width': '75px', 'height': '50px', 
                                               'max_width': '75px', 'max_height': '75px',
                                               'min_width': '50px', 'min_height': '50px'})
UnitsText = widgets.Label(value=HLUnits)
age_label = widgets.HBox([HLlabel, UnitsText])
# Link HL slider with this text
widgets.jslink((HL_slider, 'value'), (HLlabel, 'value'))

# Describe slope
slope_label = widgets.Label(value = 'Slope: {0:.2f}'.format(slope),
                                       layout={'align_items':'center','align_content':'center', 
                                               'justify_content':'center', 
                                               'width': '75px', 'height': '50px', 
                                               'max_width': '75px', 'max_height': '75px',
                                               'min_width': '50px', 'min_height': '50px'})


controls = widgets.VBox( [isotope, HL_slider, age_label, slope_label], 
                        layout=widgets.Layout(align_content='center', align_items='center', 
                                              justify_content='center', 
                                              width='300px', height='500px', 
                                              max_width='300px', max_height='500px',
                                              min_width='100px', min_height='400px',
                                              overflow_x='hidden', overflow_y='hidden') )

display(widgets.HBox( [isochron, controls] ) )


